In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip '/content/drive/MyDrive/ibm model/multiclass-weather-dataset.zip'

Archive:  /content/drive/MyDrive/ibm model/multiclass-weather-dataset.zip
  inflating: dataset/alien_test/Cloud_1.png  
  inflating: dataset/alien_test/Cloud_2.jpg  
  inflating: dataset/alien_test/Cloud_3.jpeg  
  inflating: dataset/alien_test/Cloud_4.jpg  
  inflating: dataset/alien_test/foggy_1.jpg  
  inflating: dataset/alien_test/foggy_10.jpg  
  inflating: dataset/alien_test/foggy_2.jpg  
  inflating: dataset/alien_test/foggy_3.jpg  
  inflating: dataset/alien_test/foggy_4.jpg  
  inflating: dataset/alien_test/foggy_5.jpg  
  inflating: dataset/alien_test/foggy_6.jpg  
  inflating: dataset/alien_test/foggy_7.jpg  
  inflating: dataset/alien_test/foggy_8.jpg  
  inflating: dataset/alien_test/foggy_9.jpg  
  inflating: dataset/alien_test/rain_1.jpg  
  inflating: dataset/alien_test/rain_2.png  
  inflating: dataset/alien_test/rain_3.jpg  
  inflating: dataset/alien_test/rain_4.jpg  
  inflating: dataset/alien_test/rain_5.jpg  
  inflating: dataset/alien_test/rain_6.jpg  
  inflatin

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

In [ ]:
import os

directory_path = "/content/dataset/sunrise"
count = 0

# Iterate over all the files in the directory
for filename in os.listdir(directory_path):
    # Check if the file is an image file
    if filename.endswith(".jpg") or filename.endswith(".jpeg") or filename.endswith(".png"):
        count += 1

print("Total number of images in the directory:", count)

Total number of images in the directory: 350


In [ ]:
import os
import shutil
import random

# Set the root directory containing the class folders
root_dir = "/content/dataset"

# Set the train and test directories
train_dir = "/content/train"
test_dir = "/content/test"

# Set the train-test split ratio
split_ratio = 0.8  # 80% for training, 20% for testing

# Iterate over each class folder in the root directory
for class_folder in os.listdir(root_dir):
    # Create train and test subdirectories for the class
    os.makedirs(os.path.join(train_dir, class_folder), exist_ok=True)
    os.makedirs(os.path.join(test_dir, class_folder), exist_ok=True)

    # Get a list of all the image filenames in the class folder
    image_filenames = os.listdir(os.path.join(root_dir, class_folder))
    num_images = len(image_filenames)

    # Shuffle the list of image filenames
    random.shuffle(image_filenames)

    # Split the list of image filenames into train and test sets
    split_index = int(num_images * split_ratio)
    train_filenames = image_filenames[:split_index]
    test_filenames = image_filenames[split_index:]

    # Copy the train images to the train directory
    for filename in train_filenames:
        src_path = os.path.join(root_dir, class_folder, filename)
        dst_path = os.path.join(train_dir, class_folder, filename)
        shutil.copyfile(src_path, dst_path)

    # Copy the test images to the test directory
    for filename in test_filenames:
        src_path = os.path.join(root_dir, class_folder, filename)
        dst_path = os.path.join(test_dir, class_folder, filename)
        shutil.copyfile(src_path, dst_path)

In [ ]:
# Define the input shape of the images
input_shape = (150, 150, 3)

# Define the number of classes
num_classes = 8

# Define the batch size and number of epochs for training
batch_size = 32
epochs = 50

# Define the data augmentation parameters
data_generator = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)

# Load the training data
train_data = data_generator.flow_from_directory('train', target_size=input_shape[:2], batch_size=batch_size, class_mode='categorical')

# Load the validation data
val_data = data_generator.flow_from_directory('test', target_size=input_shape[:2], batch_size=batch_size, class_mode='categorical')

# Build the CNN model
model = Sequential()

model.add(Conv2D(32, (3, 3), input_shape=input_shape, activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(train_data, epochs=epochs, validation_data=val_data)

# Save the trained model
model.save('weather_model.h5')

Found 1271 images belonging to 8 classes.
Found 353 images belonging to 8 classes.
Epoch 1/50
40/40 [==============================] - 104s 3s/step - loss: 1.4254 - accuracy: 0.4382 - val_loss: 1.0885 - val_accuracy: 0.5722
Epoch 2/50
40/40 [==============================] - 96s 2s/step - loss: 1.0756 - accuracy: 0.5964 - val_loss: 0.7849 - val_accuracy: 0.7365
Epoch 3/50
40/40 [==============================] - 96s 2s/step - loss: 0.9116 - accuracy: 0.6774 - val_loss: 0.7661 - val_accuracy: 0.7337
Epoch 4/50
40/40 [==============================] - 96s 2s/step - loss: 0.8766 - accuracy: 0.6971 - val_loss: 0.7645 - val_accuracy: 0.7479
Epoch 5/50
40/40 [==============================] - 95s 2s/step - loss: 0.7938 - accuracy: 0.7160 - val_loss: 0.6701 - val_accuracy: 0.7705
Epoch 6/50
40/40 [==============================] - 98s 2s/step - loss: 0.7845 - accuracy: 0.7207 - val_loss: 0.7146 - val_accuracy: 0.7479
Epoch 7/50
40/40 [==============================] - 96s 2s/step - loss: 0.76

In [ ]:
import os
import tensorflow as tf
from tensorflow.keras.models import load_model


model= load_model('/content/drive/MyDrive/ibm model/weather_model.h5')

# Set the path to your dataset directory
dataset_dir = "/content/dataset"

# Get the list of subdirectories (i.e., classes)
classes = os.listdir(dataset_dir)

print("Classes in dataset:", classes)

Classes in dataset: ['foggy', 'shine', 'cloudy', '.ipynb_checkpoints', 'alien_test', 'rainy', 'sunrise']


In [ ]:
from tensorflow.keras.preprocessing import image
import numpy as np

test_image = image.load_img('/content/train/sunrise/sunrise109.jpg', target_size=(150, 150))
test_image = image.img_to_array(test_image)
test_image = test_image / 255.0
test_image = np.expand_dims(test_image, axis=0)

# Resize the test image to match the input shape of the model
test_image = tf.image.resize(test_image, [150, 150])

# Make a prediction on the test image using the loaded model
prediction = model.predict(test_image)

# Get the predicted class index (the class with the highest probability)
predicted_class_index = np.argmax(prediction, axis=1)[0]

# Map the predicted class index to its corresponding class name
class_names = ['.ipynb_checkpoints','alien_test','cloudy','foggy','rainy','shine','sunrise','test.csv']
predicted_class_name = class_names[predicted_class_index]

# Print the predicted class name
print(predicted_class_name)

1/1 [==============================] - 0s 348ms/step
sunrise


In [ ]:
from tensorflow.keras.preprocessing import image

test_image = image.load_img('/content/train/rainy/rain146.jpg', target_size=(150, 150))
test_image = image.img_to_array(test_image)
test_image = test_image / 255.0
test_image = np.expand_dims(test_image, axis=0)

# Resize the test image to match the input shape of the model
test_image = tf.image.resize(test_image, [150, 150])

# Make a prediction on the test image using the loaded model
prediction = model.predict(test_image)

# Get the predicted class index (the class with the highest probability)S
predicted_class_index = np.argmax(prediction, axis=1)[0]

# Map the predicted class index to its corresponding class name
class_names = ['.ipynb_checkpoints','alien_test','cloudy','foggy','rainy','shine','sunrise','test.csv']
predicted_class_name = class_names[predicted_class_index]

# Print the predicted class name
print(predicted_class_name)

1/1 [==============================] - 0s 36ms/step
rainy


In [ ]:
from tensorflow.keras.preprocessing import image

test_image = image.load_img('/content/train/foggy/foggy108.jpg', target_size=(150, 150))
test_image = image.img_to_array(test_image)
test_image = test_image / 255.0
test_image = np.expand_dims(test_image, axis=0)

# Resize the test image to match the input shape of the model
test_image = tf.image.resize(test_image, [150, 150])

# Make a prediction on the test image using the loaded model
prediction = model.predict(test_image)

# Get the predicted class index (the class with the highest probability)
predicted_class_index = np.argmax(prediction, axis=1)[0]

# Map the predicted class index to its corresponding class name
class_names = ['.ipynb_checkpoints','alien_test','cloudy','foggy','rainy','shine','sunrise','test.csv']
predicted_class_name = class_names[predicted_class_index]

# Print the predicted class name
print(predicted_class_name)

1/1 [==============================] - 0s 61ms/step
foggy


In [ ]:
from tensorflow.keras.preprocessing import image

test_image = image.load_img('/content/test/cloudy/cloudy109.jpg', target_size=(150, 150))
test_image = image.img_to_array(test_image)
test_image = test_image / 255.0
test_image = np.expand_dims(test_image, axis=0)

# Resize the test image to match the input shape of the model
test_image = tf.image.resize(test_image, [150, 150])

# Make a prediction on the test image using the loaded model
prediction = model.predict(test_image)

# Get the predicted class index (the class with the highest probability)
predicted_class_index = np.argmax(prediction, axis=1)[0]

# Map the predicted class index to its corresponding class name
class_names = ['.ipynb_checkpoints','alien_test','cloudy','foggy','rainy','shine','sunrise','test.csv']
predicted_class_name = class_names[predicted_class_index]

# Print the predicted class name
print(predicted_class_name)

1/1 [==============================] - 0s 39ms/step
shine


In [ ]:
from tensorflow.keras.preprocessing import image

test_image = image.load_img('/content/test/shine/shine104.jpg', target_size=(150, 150))
test_image = image.img_to_array(test_image)
test_image = test_image / 255.0
test_image = np.expand_dims(test_image, axis=0)

# Resize the test image to match the input shape of the model
test_image = tf.image.resize(test_image, [150, 150])

# Make a prediction on the test image using the loaded model
prediction = model.predict(test_image)

# Get the predicted class index (the class with the highest probability)
predicted_class_index = np.argmax(prediction, axis=1)[0]

# Map the predicted class index to its corresponding class name
class_names = ['.ipynb_checkpoints','alien_test','cloudy','foggy','rainy','shine','sunrise','test.csv']
predicted_class_name = class_names[predicted_class_index]

# Print the predicted class name
print(predicted_class_name)

1/1 [==============================] - 0s 40ms/step
shine


In [ ]:
!tar -zcvf ibm_weather_classification.tgz weather_model.h5

weather_model.h5


In [ ]:
!pip install watson-machine-learning-client

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install ibm_watson_machine_learning

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from ibm_watson_machine_learning import APIClient
wml_credentials={
    "url":"https://us-south.ml.cloud.ibm.com",
    "apikey":"rmYXYMdR9dXO1ZL6v7xYC25tfVlW_ebZ4vM50d9cp423"
}

In [ ]:
client = APIClient(wml_credentials)

In [ ]:
client

In [ ]:
client.spaces.get_details()

{'resources': [{'entity': {'compute': [{'crn': 'crn:v1:bluemix:public:pm-20:us-south:a/2d933a6ee5e24c47811b7b07051c45e0:fb8cd599-a8b5-4bb2-8083-89f72c7c02fe::',
      'guid': 'fb8cd599-a8b5-4bb2-8083-89f72c7c02fe',
      'name': 'Watson Machine Learning-r1',
      'type': 'machine_learning'}],
    'description': '',
    'name': 'weather_deployment',
    'scope': {'bss_account_id': '2d933a6ee5e24c47811b7b07051c45e0'},
    'stage': {'production': False},
    'status': {'state': 'active'},
    'storage': {'properties': {'bucket_name': '79dbb3a4-7657-4654-ad84-ec3a1f1b3b44',
      'bucket_region': 'us-south',
      'credentials': {'admin': {'access_key_id': 'ba66685fcc374f9f94333a83feb9dccd',
        'api_key': 'nglPz_3pBpEHO0bLwsSSNo3wEO-oNB9O957_3sNcOy1U',
        'secret_access_key': 'd583bad2edf6bd1bcdf6aef235a276be398c60851b052dd9',
        'service_id': 'ServiceId-de599190-99f0-453f-9880-acb29b061a01'},
       'editor': {'access_key_id': '0e3f0598bc094ef08b6e4a712a9c2855',
        'a

In [ ]:
client.spaces.list()

Note: 'limit' is not provided. Only first 50 records will be displayed if the number of records exceed 50
------------------------------------  ------------------  ------------------------
ID                                    NAME                CREATED
53df13dc-442e-4f18-94b0-2fa592e07dfc  weather_deployment  2023-05-19T18:51:18.665Z
------------------------------------  ------------------  ------------------------


,ID,NAME,CREATED
0,53df13dc-442e-4f18-94b0-2fa592e07dfc,weather_deployment,2023-05-19T18:51:18.665Z


In [ ]:
space_uid = "53df13dc-442e-4f18-94b0-2fa592e07dfc"
space_uid

'53df13dc-442e-4f18-94b0-2fa592e07dfc'

In [ ]:
client.set.default_space(space_uid)

'SUCCESS'

In [ ]:
client.software_specifications.list()

----------------------------  ------------------------------------  ----  ------------------  --------------------------
NAME                          ID                                    TYPE  STATE               REPLACEMENT
default_py3.6                 0062b8c9-8b7d-44a0-a9b9-46c416adcbd9  base  unsupported         runtime-22.2-py3.10
autoai-ts_rt23.1-py3.10       01ce9391-1a79-5a33-94fb-2e134337f314  base  not_provided
kernel-spark3.2-scala2.12     020d69ce-7ac1-5e68-ac1a-31189867356a  base  not_provided
pytorch-onnx_1.3-py3.7-edt    069ea134-3346-5748-b513-49120e15d288  base  not_provided
tensorflow_rt23.1-py3.10      079a91e0-245f-5269-8926-3c20b28f37dc  base  not_provided
scikit-learn_0.20-py3.6       09c5a1d0-9c1e-4473-a344-eb7b665ff687  base  unsupported         runtime-22.2-py3.10
spark-mllib_3.0-scala_2.12    09f4cff0-90a7-5899-b9ed-1ef348aebdee  base  unsupported
pytorch-onnx_rt22.1-py3.9     0b848dd4-e681-5599-be41-b5f6fccc6471  base  create-unsupported  pytorch-onnx_rt22

,NAME,ID,TYPE,STATE,REPLACEMENT
0,default_py3.6,0062b8c9-8b7d-44a0-a9b9-46c416adcbd9,base,unsupported,runtime-22.2-py3.10
1,autoai-ts_rt23.1-py3.10,01ce9391-1a79-5a33-94fb-2e134337f314,base,not_provided,
2,kernel-spark3.2-scala2.12,020d69ce-7ac1-5e68-ac1a-31189867356a,base,not_provided,
3,pytorch-onnx_1.3-py3.7-edt,069ea134-3346-5748-b513-49120e15d288,base,not_provided,
4,tensorflow_rt23.1-py3.10,079a91e0-245f-5269-8926-3c20b28f37dc,base,not_provided,
5,scikit-learn_0.20-py3.6,09c5a1d0-9c1e-4473-a344-eb7b665ff687,base,unsupported,runtime-22.2-py3.10
6,spark-mllib_3.0-scala_2.12,09f4cff0-90a7-5899-b9ed-1ef348aebdee,base,unsupported,
7,pytorch-onnx_rt22.1-py3.9,0b848dd4-e681-5599-be41-b5f6fccc6471,base,create-unsupported,pytorch-onnx_rt22.2-py3.10
8,ai-function_0.1-py3.6,0cdb0f1e-5376-4f4d-92dd-da3b69aa9bda,base,unsupported,runtime-22.2-py3.10
9,shiny-r3.6,0e6e79df-875e-4f24-8ae9-62dcc2148306,base,not_provided,


In [ ]:
software_space_uid = client.software_specifications.get_uid_by_name("runtime-22.2-py3.10")
software_space_uid

'b56101f1-309d-549b-a849-eaa63f77b2fb'

In [ ]:
model_details = client.repository.store_model(model='/content/ibm_weather_classification.tgz', meta_props={
    client.repository.ModelMetaNames.NAME:"weather_model.h5",
    client.repository.ModelMetaNames.TYPE:"tensorflow_2.9",
    client.repository.ModelMetaNames.SOFTWARE_SPEC_UID:software_space_uid
})

In [ ]:
model_details

{'entity': {'hybrid_pipeline_software_specs': [],
  'software_spec': {'id': 'b56101f1-309d-549b-a849-eaa63f77b2fb',
   'name': 'runtime-22.2-py3.10'},
  'type': 'tensorflow_2.9'},
 'metadata': {'created_at': '2023-05-20T04:28:21.828Z',
  'id': 'f02794c5-fa4b-47b4-96ea-8c7ad3b0b048',
  'modified_at': '2023-05-20T04:28:25.249Z',
  'name': 'weather_model.h5',
  'owner': 'IBMid-661004S9GK',
  'resource_key': '899ee455-c00a-448e-b1a7-d0a8febfebb6',
  'space_id': '53df13dc-442e-4f18-94b0-2fa592e07dfc'},
 'system': {'warnings': []}}

In [ ]:
model_id = client.repository.get_model_id(model_details)
model_id

'f02794c5-fa4b-47b4-96ea-8c7ad3b0b048'

In [ ]:
client.repository.download(model_id,'CNN_weather_classification_model.tgz')

Successfully saved model content to file: 'CNN_weather_classification_model.tgz'


'/content/CNN_weather_classification_model.tgz'